# Level 3: Advanced Agentic Capabilities with Prompt Chaining and ReAct Agent

Building on the simple agent introduced in Level 2, this tutorial continues the agent-focused section of the series by introducing techniques that make the agent smarter and more autonomous: **Prompt Chaining** and the **ReAct (Reasoning + Acting) framework**. These approaches allow the agent to reason through multi-step problems, dynamically choose tools, and adjust its behavior based on context.

- **Prompt Chaining** connects multiple prompts into a coherent sequence, allowing the agent to maintain context and perform multi-step reasoning across tool invocations. [Learn more →](https://www.ibm.com/think/topics/prompt-chaining)
- **ReAct Agent** combines reasoning and acting steps in a loop, enabling the agent to make decisions, use tools dynamically, and adapt based on intermediate results. [Learn more →](https://www.ibm.com/think/topics/react-agent)

## Overview

In this notebook, you'll explore three agent configurations:
1. **Simple Agent (Baseline)** – Uses a single web search tool.
2. **Prompt Chaining** – Performs structured, multi-step reasoning by chaining prompts and responses.
3. **ReAct Agent** – Dynamically plans and executes actions using a loop of reasoning and tool use.


## Prerequisites

Before starting, ensure you have the following:
- Access to a remote cluster or a local Podman setup.
- A Tavily API key is required. You can register for one at [https://tavily.com/](https://tavily.com/).

## 1. Setting Up this Notebook
We will start with a few imports needed for this demo only.

In [1]:
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
import sys
sys.path.append('..') 
from src.client_tools import get_location

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](demos/rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [2]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)
    
print(f"Connected to Llama Stack server")

# model_id for the specific model you want to use configured with the Llama Stack server
model_id = "granite32-8b"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Connected to Llama Stack server
Inference Parameters:
	Model: granite32-8b
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 4096}
	stream: False


## 2. Simple Agent (Baseline)
Same agent setup as Level 2 notebook. 

In [3]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful websearch assistant. When you are asked to search the web you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Are there any weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'weather-related risks in current location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather-related risks in current location',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in current location',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Current', 'region': 'Harbour Island', 'country': 'Bahamas', 'lat': 25.4333, 'lon': -76.7833, 'tz_id': 'America/Nassau', 'localtime_epoch': 1746810143, 'localtime': '2025-05-09 13:02'}, 'current': {'last_updated_epoch': 1746810000, 'last_updated': '2025-05-09 13:00', 'temp_c': 24.7, 'temp_f': 76.4, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 138, 'wind_dir': 'SE', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 83, 'cloud': 16, 'feelslike_c': 26.8, 'feelslike_f': 80.3, 'windchill_c': 24.7, 'windchill_f': 76.4, 'heatindex_c': 26.8, 'heatindex_f': 80.3, 'dewpoint_c': 21.5, 'dewpoint_f': 70.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 10.9, 'gust_mph': 9.3, 'gust_kph': 14.9}}",
│   │   │   'score': 0.5844731,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Outlook for Friday, May 9 | Severe Weather Outlook',
│   │   │   'url': 'https://www.severeweatheroutlook.com/2025-05-09/',
│   │   │   'content': 'Finding your location risk... Severe Weather Outlook. show me my personal outlook → ... Day 1 Convective Outlook NWS Storm Prediction Center Norman OK 1241 AM CDT Fri May 09 2025. Valid 091200Z - 101200Z. ... ..Broyles/Lyons.. 05/09/2025. CLICK TO GET WUUS01 PTSDY1 PRODUCT. NOTE: THE NEXT DAY 1 OUTLOOK IS SCHEDULED BY 1300Z',
│   │   │   'score': 0.575517,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Hurricane Preparedness Week 2025: Know the risks in your area',
│   │   │   'url': 'https://www.wusf.org/weather/2025-05-06/hurricane-preparedness-week-2025',
│   │   │   'content': 'The start of the 2025 hurricane season is less than a month away, and there are early indicators of an active year in the tropics. Seasonal outlooks from the Climate Prediction Center forecast ENSO-neutral conditions are favored during the Northern Hemisphere summer, with a greater than 50% chance through August-October 2025.',
│   │   │   'score': 0.4948006,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'PDF',
│   │   │   'url': 'https://www.weather.gov/media/hgx/hurricanes/2025HurricaneGuide/HurricaneGuide2025.pdf',
│   │   │   'content': '2025 Southeast Texas Hurricane & Severe Weather Guide Along with big surf come strong rip currents. Rip currents are the leading surf hazard for all beach goers and result in over 100 drownings every year in the United States. The strength and size of rip currents are related to the size of the surf and wave period. Rip currents typically',
│   │   │   'score': 0.36772484,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'U.S. Hazards Outlooks - Maps | NOAA Climate.gov',
│   │   │   'url': 'https://www.climate.gov/maps-data/dataset/us-hazards-outlooks-maps',
│   │   │   'content': "Monday through Friday of each week, NOAA's Climate Prediction Center issues an outlook of weather- and climate-related hazards to the United States for the next eight to fourteen days. Potential hazards are shown for precipitation (heavy rain or snow), temperature (much-above or much-below normal), and soils (drought), Hazards maps are",
│   │   │   'score': 0.20890462,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the search results, there doesn't seem to be any immediate weather-related risks in your area that could disrupt network connectivity or system availability. 

1. The current weather in your location (Harbour Island, Bahamas) is sunny with a temperature of 76.4°F (24.7°C). There are no storms or severe weather conditions predicted for the next few hours.

2. The severe weather outlook for today (May 9, 2025) hasn't been published yet. The next outlook is scheduled by 1:00 PM CDT.

3. The 2025 hurricane season is forecasted to be active, but as of now, there are no specific warnings or advisories for your area.

4. The guide for Southeast Texas, while informative, doesn't apply to your current location.

5. NOAA's Climate Prediction Center issues weekly outlooks for weather- and climate-related hazards, but the current one doesn't show any immediate risks for your area.

Please continue to monitor local weather up

### Output Analysis

In this example, the agent hallucinates and incorrectly identifies the user's location—despite the notebook running from Dublin, Ireland. This misidentification leads to inaccurate information about potential weather-related risks.

This is where Prompt Chaining comes in. Prompt chaining allows the agent to:
1. Maintain context across multiple queries
2. Chain multiple tools together
3. Use previous interactions to inform current decisions

Let’s see how prompt chaining can improve the accuracy of the response.

## 3. Prompt chaining with websearch tool and client tool

In this section, we demonstrate a more sophisticated use case that combines location detection with web search. Instead of explicitly providing location information, we'll:

1. **Automatic Location Detection**: Use the `get_location` client tool to automatically determine the user's current location
2. **Contextual Search**: Leverage the detected location to find relevant nearby places

For example, when a user asks "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:
- First detect the user's current location
- Then use that location to search for nearby weather-related risks
- Present a comprehensive response

This demonstrates how builtin websearch tool and client tool can work together to provide intelligent, context-aware responses without requiring explicit location input from the user.

In [4]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the websearch tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Where am I?",
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("prompt-chaining-session")  # for prompt chaining, queries must share the same session_id.
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )

    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Where am I?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_location, Arguments: {'query': 'current location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Cambridge, Massachusetts, US'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
You are currently in Cambridge, Massachusetts, US.

========== Query processing completed ========== 


Processing user query: Are there any weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'weather risks in Cambridge, Massachusetts, US'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather risks in Cambridge, Massachusetts, US',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Cambridge, Massachusetts, US',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Cambridge', 'region': 'Massachusetts', 'country': 'United States of America', 'lat': 42.375, 'lon': -71.1061, 'tz_id': 'America/New_York', 'localtime_epoch': 1746809763, 'localtime': '2025-05-09 12:56'}, 'current': {'last_updated_epoch': 1746809100, 'last_updated': '2025-05-09 12:45', 'temp_c': 8.9, 'temp_f': 48.0, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 10.7, 'wind_kph': 17.3, 'wind_degree': 65, 'wind_dir': 'ENE', 'pressure_mb': 1018.0, 'pressure_in': 30.07, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 6.3, 'feelslike_f': 43.3, 'windchill_c': 8.0, 'windchill_f': 46.4, 'heatindex_c': 10.5, 'heatindex_f': 50.9, 'dewpoint_c': 9.1, 'dewpoint_f': 48.4, 'vis_km': 2.4, 'vis_miles': 1.0, 'uv': 2.4, 'gust_mph': 12.3, 'gust_kph': 19.9}}",
│   │   │   'score': 0.9509873,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'PDF',
│   │   │   'url': 'https://www.cambridgema.gov/-/media/Files/CDD/Climate/vulnerabilityassessment/finalreport_ccvapart2_mar2017_final2_web.pdf',
│   │   │   'content': 'City of Cambridge, Massachusetts. February 2017. 2. Part. ... a certain type of climate and long-term weather patterns. Cold, snowy winters, crisp autumns, wet ... access to Route 2 are also at risk. East Cambridge and Kendall Square areas are also seriously impacted',
│   │   │   'score': 0.7817479,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Cambridge, Massachusetts Climate Change Risks and Hazards ...',
│   │   │   'url': 'https://climatecheck.com/massachusetts/cambridge',
│   │   │   'content': 'The Charles watershed, which contains Cambridge, MA, has experienced 415 weeks (35% of weeks) since 2000 with some of its area in drought of any level, and 34 weeks (3% of weeks) since 2000 with some of its area in Extreme or Exceptional drought.Source: National Drought Monitor. Climate change is increasing the risk of drought.Water stress (the ratio of water demand to supply) depends on how',
│   │   │   'score': 0.76847315,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Cambridge, Massachusetts Natural Disasters | Middlesex County Hazard Db',
│   │   │   'url': 'https://hazarddb.com/region/massachusetts/cambridge-massachusetts',
│   │   │   'content': 'The county has a population of 1,629,857 and a significant build value of $305,517,292,538. Specific hazards such as hail and lightning events pose a relatively low risk, while flooding, wind, and winter weather events present a higher risk. The county also experiences moderate risks from earthquakes and wildfires.',
│   │   │   'score': 0.7327644,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Climate Change Vulnerability Assessment ... - City of Cambridge',
│   │   │   'url': 'https://www.cambridgema.gov/CDD/Projects/Climate/climatechangeresilianceandadaptation.aspx',
│   │   │   'content': "To establish a technical foundation for the Resilient Cambridge Plan, Cambridge conducted a rigorous climate change vulnerability assessment, focusing on the risks from increasing temperatures, precipitation, and sea level. The assessment sought to identify Cambridge's key physical and social vulnerabilities.",
│   │   │   'score': 0.7154444,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
According to the search results, Cambridge, Massachusetts, experiences cold, snowy winters and crisp autumns, with potential risks of flooding, wind, and winter weather events. There's also a risk of water stress due to droughts, which are increasing due to climate change. Access to Route 2 and East Cambridge, including Kendall Square areas, are particularly vulnerable. However, hazards like hail and lightning are considered to pose a relatively low risk. It's always a good idea to stay updated with local weather forecasts and advisories for the most current information.

========== Query processing completed ========== 



### ReAct Agent with websearch tool and client tool

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

For example, when asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent:
1. **Reasons** it needs location information first
2. **Acts** by calling the `get_location` client tool
3. **Observes** the location result
4. **Reasons** about the next step
5. **Acts** by calling the `websearch` tool with observed location
6. **Observes** and processes the search results

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In [5]:
agent = ReActAgent(
            client=client,
            model=model_id,
            tools=[get_location, "builtin::websearch"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
        )
user_prompts = [
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Are there any weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
{
    "thought": "I need to find out the user's current location and then check for any weather-related risks that could disrupt network connectivity or system availability. I will use the `get_location` tool to get the user's location and then the `web_search` tool to find any weather-related risks.",
    "action": {
        "tool_name": "get_location",
        "tool_params": [{"name": "query", "value": "user's current location"}]
    },
    "answer": null
}


---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Cambridge, Massachusetts, US'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Now that I have the user's location, I will use the `web_search` tool to find any weather-related risks that could disrupt network connectivity or system availability in Cambridge, Massachusetts, US.",
    "action": {
        "tool_name": "web_search",
        "tool_params": [{"name": "query", "value": "weather-related risks disrupting network connectivity or system availability in Cambridge, Massachusetts, US"}]
    },
    "answer": null
}


---------- 📍 Step 4: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather-related risks disrupting network connectivity or system availability in Cambridge, Massachusetts, US',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Cambridge, Massachusetts, US',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Cambridge', 'region': 'Massachusetts', 'country': 'United States of America', 'lat': 42.375, 'lon': -71.1061, 'tz_id': 'America/New_York', 'localtime_epoch': 1746810179, 'localtime': '2025-05-09 13:02'}, 'current': {'last_updated_epoch': 1746810000, 'last_updated': '2025-05-09 13:00', 'temp_c': 8.9, 'temp_f': 48.0, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 12.1, 'wind_kph': 19.4, 'wind_degree': 66, 'wind_dir': 'ENE', 'pressure_mb': 1018.0, 'pressure_in': 30.07, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 6.1, 'feelslike_f': 42.9, 'windchill_c': 7.2, 'windchill_f': 44.9, 'heatindex_c': 10.0, 'heatindex_f': 50.1, 'dewpoint_c': 8.7, 'dewpoint_f': 47.7, 'vis_km': 2.4, 'vis_miles': 1.0, 'uv': 2.6, 'gust_mph': 13.9, 'gust_kph': 22.4}}",
│   │   │   'score': 0.8054573,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'How Does Bad Weather Disrupt Your Internet Connectivity',
│   │   │   'url': 'https://yostream.io/blog/bad-weather-disrupts-internet-connectivity/',
│   │   │   'content': 'Directly, lightning can strike network infrastructure such as cell towers, data centers, or internet exchanges, causing physical damage and disrupting connectivity. Severe weather conditions like hurricanes, floods, or landslides can cause physical damage to the fiber optic cables, resulting in outages or disruptions to internet connectivity. In rare cases, power outages caused by bad weather can also impact fiber optic internet connectivity, particularly if the internet service provider’s network infrastructure relies on power-dependent equipment. Internet outages and downtime can be caused by various weather-related factors, including power outages, physical damage to infrastructure, and flooding that affects data centers and network equipment. Heavy rainfall, thunderstorms, and other severe weather events can exacerbate network congestion by disrupting the infrastructure that supports internet connectivity.',
│   │   │   'score': 0.4395795,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'How Bad Weather Impacts Different Types of Internet: Fiber, Cable, 5G ...',
│   │   │   'url': 'https://www.geneseo.com/internet-bad-weather/',
│   │   │   'content': "Inclement weather can interfere with your plans and special occasions, causing delays or cancellations. Beyond mere inconvenience, bad weather can significantly hinder various forms of internet connectivity. Whether it's heavy rain, snow, severe thunderstorms, or high winds, different types of internet connections react in unique ways.",
│   │   │   'score': 0.18374038,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Can a Storm Knock Out My Internet? Understanding the Connection Between ...',
│   │   │   'url': 'https://softhandtech.com/can-a-storm-knock-out-my-internet/',
│   │   │   'content': 'The duration of an internet outage depends on several factors, including the extent of the storm’s damage and the speed of service restoration efforts. Can a storm really disrupt my internet connection? Lastly, following your internet service provider’s updates and alerts regarding potential outages during severe weather can help you stay informed about your connectivity status during a storm. Most internet service providers (ISPs) have protocols in place for storm-related outages and typically provide support to affected customers. Yes, satellite internet services can be more vulnerable to storms compared to other types of internet connections due to their reliance on satellite signals. To determine 


---------- 📍 Step 5: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Based on the search results, there are several ways weather can disrupt network connectivity or system availability. These include direct strikes from lightning, physical damage to infrastructure from severe weather conditions, power outages, and network congestion. However, the current weather forecast for Cambridge, Massachusetts, US does not indicate any immediate or severe weather events that could cause such disruptions.",
    "action": null,
    "answer": "Based on the current weather forecast, there are no immediate or severe weather events in Cambridge, Massachusetts, US that could disrupt network connectivity or system availability. However, severe weather conditions like hurricanes, floods, or heavy rainfall could potentially cause such disruptions."
}

========== Query processing completed ========== 



## Key Takeaways
- This tutorial demonstrates how to build smarter agents using Prompt Chaining and the ReAct framework.
- It shows how agents can maintain context across steps and perform structured, multi-step reasoning.
- It highlights how ReAct enables dynamic tool selection and adaptive decision-making based on intermediate results.
- These techniques enhance agent autonomy and make them more suitable for complex operational tasks.

For further extensions, continue exploring the next levels in this notebook series.